In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
# We create our model
model = gp.Model('Promotions')


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-05-18
Using license file C:\Users\aishw\gurobi.lic


In [3]:
#sales = [90, 89, 11, 143, 103, 154, 116, 207, 133, 133, 134, 131, 168, 200, 105]
disc = [1, 0.9, 0.8, 0.6]
demand_rate = [1, 1.05, 1.1, 1.2]
price = 60
initial_demand = 50
units = 2000
#contract_upper_threshold = pd.read_excel(file_loc,sheet_name='Contract Upper Threshold',header=2, na_values=['NA'], usecols="B:F")

In [4]:
#ind = [(i,j) for i in range(15) for j in range(4)]

p = model.addVars(15, vtype = GRB.CONTINUOUS, name = ["price_"+str(i) for i in range(1,16)])
pd = model.addVars(15, 4, vtype = GRB.BINARY, name = "decide_price_")
d = model.addVars(15, vtype = GRB.CONTINUOUS, name = ["demand_"+str(i) for i in range(1,16)])
y = model.addVars(15, lb = -GRB.INFINITY,vtype = GRB.INTEGER, name = ["bounding_demand_"+str(i) for i in range(1,16)])

In [5]:
#revenue = sum(D[m,z]*r.iloc[0,z] for m in range(1, M) for z in range(Z))

rev = sum(p[i]*y[i] for i in range(15))

model.setObjective(rev, GRB.MAXIMIZE)

In [6]:
model.addConstr(d[0]==initial_demand)
model.addConstr(y[0]==initial_demand)

for i in range(15):
    model.addConstr(sum(pd[i,j] for j in range(4))==1, name = "Only_one_discount_"+str(i))
    
for i in range(14):
    model.addConstr(sum(pd[i,j]*disc[j]*price for j in range(4))>=sum(pd[i+1,j]*disc[j]*price for j in range(4)), name = "maintain_or_reduce_discount_"+str(i))
    model.addConstr(d[i+1]==(sum(pd[i,j]*demand_rate[j]*d[i] for j in range(4))-(1.5*i+1)), name = "pred_demand_"+str(i))
    model.addConstr(p[i+1]==sum(pd[i+1,j]*disc[j]*price for j in range(4)), name = "price_"+str(i))
    model.addConstr(d[i+1]-0.999<=y[i+1], name = "demand_bound_low_"+str(i))
    model.addConstr(y[i+1]<=d[i+1], name = "demand_bound_high_"+str(i))
    

In [7]:
model.addConstr(sum(d[i] for i in range(15))<=units+2, name = "Total_stock")
model.addConstr(sum(y[i] for i in range(15))<=units, name = "Total_stock_bounding")
model.addConstr(pd[0,0]==1)
model.addConstr(p[0]==60)

<gurobi.Constr *Awaiting Model Update*>

In [8]:
model.params.NonConvex = 2
model.optimize()

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 77 rows, 105 columns and 332 nonzeros
Model fingerprint: 0xddc5e0c3
Model has 15 quadratic objective terms
Model has 14 quadratic constraints
Variable types: 30 continuous, 75 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
  QRHS range       [1e+00, 2e+01]
Presolve removed 8 rows and 12 columns
Presolve time: 0.00s
Presolved: 252 rows, 155 columns, 806 nonzeros
Presolved model has 13 bilinear constraint(s)
Variable types: 75 continuous, 80 integer (54 binary)

Root relaxation: objective 1.199954e+05, 198 iterations, 0.00

In [9]:
if model.status == GRB.OPTIMAL:
    model.printAttr('X')


    Variable            X 
-------------------------
     price_1           60 
     price_2           36 
     price_3           36 
     price_4           36 
     price_5           36 
     price_6           36 
     price_7           36 
     price_8           36 
     price_9           36 
    price_10           36 
    price_11           36 
    price_12           36 
    price_13           36 
    price_14           36 
    price_15           36 
decide_price_[0,0]            1 
decide_price_[1,3]            1 
decide_price_[2,3]            1 
decide_price_[3,3]            1 
decide_price_[4,3]            1 
decide_price_[5,3]            1 
decide_price_[6,3]            1 
decide_price_[7,3]            1 
decide_price_[8,3]            1 
decide_price_[9,3]            1 
decide_price_[10,3]            1 
decide_price_[11,3]            1 
decide_price_[12,3]            1 
decide_price_[13,3]            1 
decide_price_[14,3]            1 
    demand_1           50 
    demand_2  

In [10]:
model.write('model.lp')

In [11]:
def sen_report(model):
    print('Sensitivity Analysis (SA)\n ObjVal =', model.ObjVal)
    #model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])
    #model.printAttr(['X', 'RC', 'LB', 'SALBLow', 'SALBUp', 'UB', 'SAUBLow', 'SAUBUp'])
    model.printAttr(['Sense', 'Slack', 'Pi', 'RHS', 'SARHSLow', 'SARHSUp']) # Pi = shadow price
    # NOTE: printAttr prints only rows with at least one NON-ZERO value, e.g. model.printAttr('X') prints only non-zero variable values

In [12]:
sen_report(model)

Sensitivity Analysis (SA)
 ObjVal = 50412.000000016706

  Constraint        Sense        Slack           Pi          RHS     SARHSLow      SARHSUp 
------------------------------------------------------------------------------------------


GurobiError: Unable to retrieve attribute 'Pi'